# Superdense Coding

#### Used for sending 2 classical bits using one qubit.

#### Eve creates the Bell state and sends 1 qubit each to Alice and Bob.
Alice applies some operations to her qubit based on the 2 classical bits she wants to send: (first bit represents Bob's qubit and second - Alice's)

| Intended Message | Applied Gate | Resulting State|
|:----------------:|:------------:|:--------------:|
|$00$             |$I$           |$|00\rangle + |11\rangle$|
|$10$             |$X$           |$|01\rangle + |10\rangle$|
|$01$             |$Z$           |$|00\rangle - |11\rangle$|
|$11$             |$ZX$          |$|10\rangle - |01\rangle$|


Alice then sends her qubit to Bob, who applies some gates: (first bit represents Bob's qubit and second - Alice's).\
For the CX gates, the second qubit (Alice's) acts as the control and the first acts as the target.

| Bob Receives            | After CNOT Gate           | After Hadamard Gate|
|:-----------------------:|:-------------------------:|:------------------:|
|$|00\rangle + |11\rangle$|  $|00\rangle + |01\rangle$| $|00\rangle$       |
|$|01\rangle + |10\rangle$|  $|11\rangle + |10\rangle$| $|10\rangle$       |
|$|00\rangle - |11\rangle$|  $|00\rangle - |01\rangle$| $|01\rangle$       |
|$|10\rangle - |01\rangle$|  $|10\rangle - |11\rangle$| $|11\rangle$       |

## Code

In [ ]:
from qiskit import Aer, QuantumCircuit, execute, IBMQ
from qiskit.visualization import plot_histogram as plot_h
%config InlineBackend.figure_format = 'svg'

In [ ]:
out = 'mpl'
backend = Aer.get_backend('qasm_simulator')
shots = 1024

In [ ]:
def create_bell_pair(qc, a, b):
    qc.h(a)
    qc.cx(a,b)

In [ ]:
def encode_msg(qc, qubit, msg):
    assert msg in ['00', '01', '10', '11'], "Invalid message"
    if msg == '00':
        pass
    elif msg == '10':
        qc.x(qubit)
    elif msg == '01':
        qc.z(qubit)
    elif msg == '11':
        qc.z(qubit)
        qc.x(qubit)   

In [ ]:
def decode_msg(qc, a, b):
    qc.cx(a, b)
    qc.h(a)

In [ ]:
qc = QuantumCircuit(2)

alice = 0
bob = 1

create_bell_pair(qc, alice, bob)
qc.barrier()

# Assume, at this point, that qubit 0 goes to Alice and qubit 1 goes to Bob

msg = '10'
encode_msg(qc, alice, msg)
qc.barrier()

decode_msg(qc, alice, bob)

qc.measure_all()

qc.draw(out)

In [ ]:
res = execute(qc, backend).result()
counts = res.get_counts()
plot_h(counts)

#### Executing on a real backend to measure accuracy

In [ ]:
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
shots = 256

In [ ]:
IBMQ.load_account()

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
backend = least_busy(provider.backends(filters = lambda x: x.configuration().n_qubits >= 2 
                                       and not x.configuration().simulator 
                                       and x.status().operational == True))
print(backend)

In [ ]:
job = execute(qc, backend=backend, shots=256)

In [ ]:
job_monitor(job)

In [ ]:
res = job.result()
plot_h(res.get_counts())

In [ ]:
correct_res = res.get_counts()[msg]
accuracy = correct_res*100/shots
print("Accuracy = %.2f%%" % accuracy)